<a href="https://colab.research.google.com/github/drv-j/ACMDIGW/blob/ML/extracting_and_saving_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# most common short-cut keys
# ctrl + / = to comment a code
# ctrl + enter = run a code block
# ctrl + m, b = reate a code block below
# ctrl + m, a = cretae a code block above

In [ ]:
#connect google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install yelpapi
!pip install yelpapi --quiet

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# additional libraries
import os,json,math,time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

Loading Credentials and Creating the YelpAPI Object

In [ ]:
#load yelp API credentials and instamtiate the YelpAPI object
with open('/content/drive/MyDrive/Colab Notebooks/Credentials/yelp_api.json') as file:
  yelp_credentials = json.load(file)
# create the yelp api engine
yelp_api = YelpAPI(yelp_credentials['api-key'], timeout_s=5.0)

In [ ]:
#define API call parameters and output file path
LOCATION = 'Greenville, SC'
TERM = 'Sushi'
JSON_FILE = '/Data/results_SC_sushi.json'

# display the destination file path
print(f'Data will be saved to: {JSON_FILE}')

Data will be saved to: /Data/results_SC_sushi.json


In [ ]:
#check if JSON_FILE exists and create if it doesn't
if not os.path.isfile(JSON_FILE):

  #create the directory
  os.makedirs(os.path.dirname(JSON_FILE), exist_ok = True)

  # confirm and safe an empty list on the file
  print(f'[i] {JSON_FILE} not found. Saving empty list on file')
  with open (JSON_FILE, 'w') as file:
    json.dump([],file)

else :
  # inform the user if the file already exists
  print(f'[i] {JSON_FILE} already exists.')

[i] /Data/results_SC_sushi.json already exists.


In [ ]:
# load previous results and set offset based on the number of results
with open(JSON_FILE, 'r') as file:
  previous_results = json.load(file)

  n_results = len(previous_results)

  print(f'-{n_results} previous results found.')

-20 previous results found.


In [ ]:
# make first API call
results = yelp_api.search_query(location = LOCATION,
                                term = TERM,
                                offset = n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [ ]:
#return how many results were found
total_results = results ['total']
total_results

111

In [ ]:
business_results = results['businesses']
#business_dara was replaced with business_results

# specify the filename where you want to save the data
json_file_path = JSON_FILE

# save the business results
with open (json_file_path, 'w') as file:
  json.dump(business_results, file, indent = 4)

In [ ]:
#how many details did we get?
results_per_page = len(business_results)
# business data was replaced with business_results
print(f'number of results retrieved per page',results_per_page)

number of results retrieved per page 20


In [ ]:
#using the math.ceil to the round up the total number of pages
n_pages = math.ceil(total_results/results_per_page)
print(f'total number of pages: {n_pages}')

total number of pages: 6


In [ ]:
#create a lopp to extract the data
for i in tqdm_notebook(range(1, total_results + 1)):  #makes a loop
  try:
    time.sleep(0.2) #short delay to respect API rata limits
    #so that when we call,pause, call pause, it doesn't think we are attacking it

    #load existing results to append new data
    with open(JSON_FILE, 'r') as file:
      previous_results = json.load(file)

    #fetch new results
    new_results = yelp_api.search_query(location = LOCATION,
                                        term = TERM,
                                        offset=len(previous_results))
    #append and save the updated results
    updated_results = previous_results + new_results ['businesses']
    with open(JSON_FILE, 'w') as file: # w should not be captial
      json.dump(updated_results, file)

  except Exception as e:
    if 'Too Many Requests for url'in str(e):
      print('Rare limit exceeded. Stoppin data-collection')
      break

    else: #if there is an error, print it and continue if able
      print(f'An error occured: {e}')
      continue

  0%|          | 0/111 [00:00<?, ?it/s]

In [ ]:
# load the final JSON file
df = pd.read_json (JSON_FILE)

#display the first 5 rows of the data
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,HlMRxvnD7jC6uMUYZCsDCg,tsunami-japanese-sushi-bar-and-grill-greenville,Tsunami Japanese Sushi Bar and Grill,https://s3-media1.fl.yelpcdn.com/bphoto/w5Q5tO...,False,https://www.yelp.com/biz/tsunami-japanese-sush...,139,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.5,"{'latitude': 34.85197, 'longitude': -82.39704}","[pickup, delivery]",$$,"{'address1': '106 E North St', 'address2': '',...",+18644671055,(864) 467-1055,3635.170753
1,SslytQ5fGcY7DN5a5vRi5A,hibachi-cafe-greenville,Hibachi Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/AYwmKG...,False,https://www.yelp.com/biz/hibachi-cafe-greenvil...,81,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 34.8211089, 'longitude': -82.3403...","[pickup, delivery]",$$,"{'address1': '101 Verdae Blvd', 'address2': 'S...",+18642888595,(864) 288-8595,3341.398427
2,0NNl-TjwwDkRl0oAZK4p3A,sushi-asa-greenville,Sushi Asa,https://s3-media1.fl.yelpcdn.com/bphoto/xEYZ7_...,False,https://www.yelp.com/biz/sushi-asa-greenville?...,145,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.2,"{'latitude': 34.8171189, 'longitude': -82.2755...","[pickup, delivery]",$$,"{'address1': '150 Tanner Rd', 'address2': None...",+18642885259,(864) 288-5259,8392.529884
3,pCpt-b8_ashf7-lVwO8uBQ,zen-ramen-and-sushi-burrito-greenville,Zen Ramen and Sushi Burrito,https://s3-media1.fl.yelpcdn.com/bphoto/yXFBCq...,False,https://www.yelp.com/biz/zen-ramen-and-sushi-b...,45,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.4,"{'latitude': 34.8207592, 'longitude': -82.2636...",[delivery],NaN,"{'address1': '1939-B Woodruff Rd', 'address2':...",+18646094066,(864) 609-4066,9082.998384
4,vILtQZOAP_LqkV_BB4e0cQ,chef-21-sushi-burger-simpsonville-2,Chef 21 Sushi Burger,https://s3-media2.fl.yelpcdn.com/bphoto/XJDcDW...,False,https://www.yelp.com/biz/chef-21-sushi-burger-...,90,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.3,"{'latitude': 34.70605268375681, 'longitude': -...","[pickup, delivery]",NaN,"{'address1': '699 Fairview Rd', 'address2': 'S...",+18646882198,(864) 688-2198,18283.027592


In [ ]:
# Specify the directory
directory = '/content/drive/MyDrive/Colab Notebooks/Data'
filename = 'final_results_SC_suhsi.csv.gz' # make sure to include the .csz.gz extension
path = os.path.join(directory, filename)

# ensure that the 'Data' directory exists
os.makedirs(directory, exist_ok=True)

# save the dataframe as a compressed csv file (to save space)
df.to_csv(path, compression='gzip', index=False)
# df was defined in the previous code block: df = pd.read_json(JSON_FILE)

In [ ]:
# save as JSON file
json_file = '/content/dirve/MyDrive/Colab Notebooks/Data/final_results_SC_sushi.json'
# add Data folders copied path before / final_results_SC_sushi.json

# save the json as a dataframe
df.to_json(json_file, orient='records', lines=True)

In [ ]:
# save the file as gzip
csv_gz_file = json_file.replace('.json', '.csv.gz')

# save the dataframne as a compressed csv without the index
df.to_csv(csv_gz_file, compression='gzip',index=False)